In [4]:
import numpy as np
import matplotlib.pyplot as plt
import vector
import pandas as pd

In [25]:
# Cabbibo angle, from https://en.wikipedia.org/wiki/Cabibbo%E2%80%93Kobayashi%E2%80%93Maskawa_matrix
theta_c = 13.02 * np.pi/180 # radians
# Tau mass
mtau = 1.77682 # GeV
#
M_V = 0.84 # GeV
# difference between muon and electron magnetic moments (I think, see tau polarization paper)
xi = 3.706
# 
M_A = 1 # GeV
#
F_A_0 = -1.23
# Pion mass
m_pi = 0.139 # GeV

def sz(tau4m, nu4m, nucleon4m, int_type: str):
    M = nucleon4m.m
    ptau = tau4m.p
    Etau = tau4m.E
    costheta = nu4m.deltaangle(tau4m) # TODO this might not work for 4-vectors
    Enu = nu4m.E
    p = nucleon4m
    k = nu4m
    kprime = tau4m
    
    q = k - kprime # W boson 4-momentum
    # I might also pass this as a parameter. Thoug probably not, since the model assumes that it is a nucleon, not a nucleus.
    Q2 = -q**2 # TODO check that this actually gives a positive scalar.
    x = Q2 / (2*p*q) # Björken scaling variable. TODO check that this actually gives a scalar

    F = (
        (2*W1(x, Q2, p, q, M, int_type) + mtau**2 / M**2 * W4(x, Q2, p, q, M, int_type)) * (Etau - ptau * costheta)
        + W2(x, Q2, p, q, M, int_type) * (Etau + ptau * costheta)
        + W3(x, Q2, p, q, M, int_type) / M * (Enu * Etau + ptau**2 - (Enu + Etau) * ptau * costheta)
        + mtau**2 / M * W5(x, Q2, p, q, M, int_type)
    )

    return 1/2 * (
        (2*W1(x, Q2, p, q, M, int_type) - mtau**2 / M**2 * W4(x, Q2, p, q, M, int_type)) * (ptau - Etau * costheta)
        + W2(x, Q2, p, q, M, int_type) * (ptau + Etau * costheta)
        + W3(x, Q2, p, q, M, int_type) / M * ((Enu + Etau) * ptau - (Enu * Etau + ptau**2) * costheta)
        - mtau**2 / M * W5(x, Q2, p, q, M, int_type) * costheta
    ) / F


def W1(x, Q2, p, q, M, int_type):
    if int_type == "qel":
        return W1qel(x, Q2, p, q, M)
    elif int_type == "res":
        return W1res(x, Q2, p, q, M)
    elif int_type == "dis":
        return W1dis(x, Q2, p, q, M)
    else:
        raise ValueError("Invalid interaction type")
    
def W2(x, Q2, p, q, M, int_type):
    if int_type == "qel":
        return W2qel(x, Q2, p, q, M)
    elif int_type == "res":
        return W2res(x, Q2, p, q, M)
    elif int_type == "dis":
        return W2dis(x, Q2, p, q, M)
    else:
        raise ValueError("Invalid interaction type")
    
def W3(x, Q2, p, q, M, int_type):
    if int_type == "qel":
        return W3qel(x, Q2, p, q, M)
    elif int_type == "res":
        return W3res(x, Q2, p, q, M)
    elif int_type == "dis":
        return W3dis(x, Q2, p, q, M)
    else:
        raise ValueError("Invalid interaction type")
    
def W4(x, Q2, p, q, M, int_type):
    if int_type == "qel":
        return W4qel(x, Q2, p, q, M)
    elif int_type == "res":
        return W4res(x, Q2, p, q, M)
    elif int_type == "dis":
        return W4dis(x, Q2, p, q, M)
    else:
        raise ValueError("Invalid interaction type")
    
def W5(x, Q2, p, q, M, int_type):
    if int_type == "qel":
        return W5qel(x, Q2, p, q, M)
    elif int_type == "res":
        return W5res(x, Q2, p, q, M)
    elif int_type == "dis":
        return W5dis(x, Q2, p, q, M)
    else:
        raise ValueError("Invalid interaction type")

def w(p, q, M):
    return p*q/M**2

def G_V_E(q):
    return 1 / (1-q**2/M_V**2)**2

def G_V_M(q):
    return (1+xi) / (1-q**2/M_V**2)**2

def F_V(q, M):
    return (G_V_E(q) - q**2 / (4*M**2) * G_V_M(q)) / (1 - q**2 / (4*M**2))

def F_A(q):
    return F_A_0 / (1 - q**2 / M_A**2)**2

def F_M(q, M):
    return (G_V_M(q) - G_V_M(q)) / (xi * (1 - q**2 / (4*M**2)))

def F_T(q):
    return 0

def F_S(q):
    return 0

def F_p(q, M):
    return 2*M**2 * F_A(q) / (m_pi**2 - q**2)

def Wqel_coefficient(x, Q2, p, q, M):
    assert np.isclose(x, 1)  # a delta function here where x must be 1
    return np.cos(theta_c)**2 * 1/w(p, q, M) 

def W1qel(x, Q2, p, q, M):
    xprime = Q2 / (4*M**2)
    return (
        Wqel_coefficient(x, Q2, p, q, M)
        * (F_A(q)**2 + xprime * (F_A(q)**2 + (F_V(q, M) + F_M(q))**2))
    )

def omega_2(x, Q2, p, q, M):
    xprime = Q2 / (4*M**2)
    return F_V(q)**2 + F_A(q)**2 + xprime * (F_M(q)**2 + 4*F_T(q)**2)

def W2qel(x, Q2, p, q, M):
    assert np.isclose(x, 1)
    return (
        Wqel_coefficient(x, Q2, p, q, M)
        * omega_2(x, Q2, p, q, M)
    )

def W3qel(x, Q2, p, q, M):
    return (
        Wqel_coefficient(x, Q2, p, q, M)
        * -2 * np.real(np.conj(F_A(q)) * (F_V(q) + F_M(q)))
    )

def W4qel(x, Q2, p, q, M):
    xprime = Q2 / (4*M**2)
    return (
        Wqel_coefficient(x, Q2, p, q, M)
        # omega_4
        * (
            np.real(np.conj(F_V(q)) * (F_S(q) - 1/2 * F_M(q)) - np.conj(F_A(q)) * (F_T(q) + F_p(q, M)))
            + xprime * (1/2 * (F_M(q) - F_S(q))**2 + (F_T(q) + F_p(q, M))**2)
            - 1/4* (1+xprime) * F_M(q)**2 + (1+1/2 * xprime) * F_S(q)**2
        )
    )

def W5qel(x, Q2, p, q, M):
    xprime = Q2 / (4*M**2)
    return (
        Wqel_coefficient(x, Q2, p, q, M)
        * (
            2 * np.real(np.conj(F_S(q)) * (F_V(q, M) - xprime*F_M(q, M)) - np.conj(F_T) * (F_A(q, M) - 2*xprime*F_p(q, M)))
            + omega_2(x, Q2, p, q, M)
        )
    )

# Delta mass
M_delta = 1.232 # GeV
# Delta width assuming dominance of S-waves
Gamma_delta = 0.12 # GeV
# Is 2 if the Delta produced is Delta++ (interaction with p), 
# and 2/3 if the Delta produced is Delta+ (interaction with n). 
# Maybe take average, since there are equally many n and p in Oxygen?
kappa = 2

# def W1res(x, Q2, p, q, M):
#     return (
#         kappa * np.cos(theta_c)**2 * M * M_delta * 
#         eta_delta_bw(W)**2 * 
#         sum(
#             V_1(x, Q2, j, k) * C_V(x, Q2, j) * C_V(x, Q2, k) 
#             + A_1(x, Q2, j, k) * C_A(x, Q2, j) * C_A(x, Q2, k) 
#             for j, k in itertools.product(range(3, 7), repeat=2)
#         )
#     )

In [26]:
particle_info = pd.read_csv("../data/test_genie_NuTau_10.0_GeV_particles.csv")
event_info = pd.read_csv("../data/test_genie_NuTau_10.0_GeV_event_info.csv")

In [27]:
qel_particles = particle_info[
    particle_info["event_num"]
    .isin(event_info.loc[event_info["qel"], "event_num"].values)
]

In [28]:
qel_particles

,event_num,pdg,E,px,py,pz,polx,poly,polz
25,4,1000080160,14.895082,0.000000,0.000000,0.000000,NaN,NaN,NaN
26,4,16,10.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000
27,4,15,9.600037,0.610560,-0.685543,9.389402,-0.072666,-0.064718,-0.995254
28,4,-211,1.324248,0.114591,-0.388702,1.252969,NaN,NaN,NaN
29,4,-211,0.951341,0.040753,0.362281,0.867560,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
258470,42482,16,10.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000
258471,42482,15,9.717707,0.501494,0.550021,9.524848,0.057570,-0.052491,-0.996961
258472,42482,13,5.608811,-0.528947,0.263967,5.576570,NaN,NaN,NaN
258473,42482,-14,2.656325,0.725682,0.009534,2.555261,NaN,NaN,NaN


In [29]:

nucleon4m = vector.obj(**{"E": 0.938, "px": 0, "py": 0, "pz": 0})
nu4m = vector.obj(**{"E": 10, "px": 0, "py": 0, "pz": 10})
tau4m = vector.obj(**{"E": qel_particles.iloc[2, 2], "px": qel_particles.iloc[2, 3], "py": qel_particles.iloc[2, 4], "pz": qel_particles.iloc[2, 5]})
tau4m

MomentumObject4D(px=0.6105601607251381, py=-0.6855431035549308, pz=9.3894023670702, E=9.600037457210252)

In [30]:
sz(tau4m, nu4m, nucleon4m, "qel")

TypeError: operand type(s) all returned NotImplemented from __array_ufunc__(<ufunc 'multiply'>, '__call__', MomentumObject4D(px=0, py=0, pz=0, E=1.876), MomentumObject4D(px=-0.6105601607251381, py=0.6855431035549308, pz=0.6105976329297995, E=0.39996254278974774)): 'MomentumObject4D', 'MomentumObject4D'

In [24]:
np.sqrt(tau4m**2)

1.776819999999989